## Model Deployment with Spark Serving 
In this example, we try to movie recommendations from the *Movie Ratings* dataset. Then we will use Spark serving to deploy it as a realtime web service. 
First, we import needed packages:

In [2]:
import sys
import numpy as np
import pandas as pd
import mmlspark
import os

from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

Now let's read the data and split it to train and test sets:

In [4]:
#Columns
userCol='UserId'
itemCol='MovieId'
ratingCol='Rating'

# Download Movie Lens
basedataurl = "http://aka.ms" 
datafile = "MovieRatings.csv"

datafile_dbfs = os.path.join("/dbfs", datafile)

if os.path.isfile(datafile_dbfs):
    print("found {} at {}".format(datafile, datafile_dbfs))
else:
    print("downloading {} to {}".format(datafile, datafile_dbfs))
    urllib.request.urlretrieve(os.path.join(basedataurl, datafile), datafile_dbfs)
    
data_all = sqlContext.read.format('csv')\
                     .options(header='true', delimiter=',', inferSchema='true', ignoreLeadingWhiteSpace='true', ignoreTrailingWhiteSpace='true')\
                     .load(datafile).cache()    
data_all.printSchema()
display(data_all)

Next, we will create a Cross Validator pipeline, in order to tune a Spark ALS model.

In [6]:
from mmlspark.RankingTrainValidationSplit import RankingTrainValidationSplit
from mmlspark.RankingEvaluator import RankingEvaluator

als = ALS() \
    .setUserCol(userCol) \
    .setRatingCol(ratingCol) \
    .setItemCol(itemCol) \

paramGrid = ParamGridBuilder() \
    .addGrid(als.maxIter, [2, 4, 8]) \
    .addGrid(als.regParam, [1, 0.1, 0.01]) \
    .build()

itemCount = data_all.select(itemCol).distinct().count()
evaluator = RankingEvaluator()

rankingTrainValidationSplit = RankingTrainValidationSplit() \
    .setEstimator(als) \
    .setEvaluator(evaluator) \
    .setEstimatorParamMaps(paramGrid) \
    .setTrainRatio(0.8) \
    .setCollectSubMetrics(True)

model = rankingTrainValidationSplit.fit(data_all)

recommendations = model.bestModel.recommendForAllUsers(10).cache()

Now, we will define the webservice input/output.
For more information, you can visit the [documentation for Spark Serving](https://github.com/Azure/mmlspark/blob/master/docs/mmlspark-serving.md)

In [8]:
from pyspark.sql.functions import col, from_json, broadcast
from pyspark.sql.types import *
import uuid
from mmlspark import request_to_string, string_to_response

serving_inputs = spark.readStream.server() \
    .address("localhost", 8898, "my_api") \
    .load()\
    .parseRequest(data_all.select(userCol).schema)

serving_outputs = serving_inputs \
  .join(broadcast(recommendations), userCol) \
  .makeReply("recommendations")

server = serving_outputs.writeStream \
    .server() \
    .replyTo("my_api") \
    .queryName("my_query") \
    .option("checkpointLocation", "checkpoints-{}".format(uuid.uuid1())) \
    .start()


Test the webservice

In [10]:
import requests
data = u'{"UserId":1}'
r = requests.post(data=data, url="http://localhost:8898/my_api")
print("Response {}".format(r.text))

In [11]:
import requests
data = u'{"UserId":200}'
r = requests.post(data=data, url="http://localhost:8898/my_api")
print("Response {}".format(r.text))

In [12]:
import time
time.sleep(20) # wait for server to finish setting up (just to be safe)
server.stop()